In [12]:
import pandas as pd
import anndata
import scanpy as sc
from SCCEE import *
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import glob
import multiprocessing
import functools
import sys

In [13]:
from hba_data import internal_utils as hba_utils

In [14]:
cell_meta = hba_utils.CellType.get_mc_meta()
cell_meta['Region'] = hba_utils.Region.correct_region(cell_meta['Region'])
cell_meta['SupRegion'] = hba_utils.Region.get_brain_structure(cell_meta['Region'])

In [15]:
cell_meta = cell_meta[(cell_meta['MajorType']!='Outline')&(cell_meta['SubType']!='Outline')]

In [16]:
sel1 = (cell_meta['Donor']=='h1930004')&(cell_meta['Region']=='A19')
sel2 = (cell_meta['Donor']=='h1930004')&(cell_meta['Region']=='LEC')
cell_meta.loc[sel1, 'Region'] = 'LEC'
cell_meta.loc[sel2, 'Region'] = 'A19'

In [17]:
# def worker_base(n_nbrs, sccee, cond, n_pseudos, ct, obsm):
#     print(ct, n_nbrs, 'start')
#     sys.stdout.flush()
#     rlt = sccee.evaluate_conditional_effect(cond, n_pseudos=n_pseudos, 
#                                             n_nbrs=n_nbrs, use_cached_bg=True)
#     joblib.dump(rlt, f'./regional-axes-data/{ct}.{obsm}.nbr_{str(n_nbrs).zfill(3)}.sccee.joblib') 
#     print(ct, n_nbrs, 'done')
#     sys.stdout.flush()
#     return True

In [18]:
done = set([x.split('/')[-1].split('.X_pca_scanorama.npseudos_')[0] for x in glob.glob('regional-axes-data/*')])

In [19]:
todo = set([x.split('/')[-2] for x in glob.glob('/gale/netapp/home/wtian/hba/analysis/CellTypeMergingAndSummary/CellTypeEmbedding/MajorType-type-region-selected/*.*/adata.with_coords.h5ad')])


In [20]:
todo-=done

In [21]:
done

{'CHD7.BN_BF',
 'Foxp2.BN_BF',
 'L23_IT.CX',
 'L4_IT.CX',
 'L5_IT.CX',
 'L6_IT.CX',
 'Lamp5.CX',
 'Lamp5.CX_HIP',
 'Lamp5_LHX6.CX',
 'Lamp5_LHX6.CX_HIP',
 'MSN_D1.BN_BF',
 'MSN_D2.BN_BF',
 'Pvalb.CX',
 'Pvalb.CX_HIP',
 'Pvalb_ChC.CX',
 'Pvalb_ChC.CX_HIP',
 'Sncg.CX',
 'Sncg.CX_HIP',
 'Sst.CX',
 'Sst.CX_HIP',
 'Vip.CX',
 'Vip.CX_HIP'}

In [22]:
todo

{'L56_NP.CX', 'L6_CT.CX', 'L6b.CX'}

In [23]:
for fn in glob.glob(f'/gale/netapp/home/wtian/hba/analysis/CellTypeMergingAndSummary/'
                    f'CellTypeEmbedding/MajorType-type-region-selected/*.*/adata.with_coords.h5ad'):
    n_nbrs = 200
    n_pseudos = 'all'
    
    ct,supregs = fn.split('/')[-2].split('.')
#     if f'{ct}.{supregs}' in done:
#         continue
    if f'{ct}.{supregs}' not in todo:
        continue    
#     if ct!='L4_IT' or supregs!='CX':
#         continue

#     supregs = supregs.split('_')

    print(ct,supregs)
    adata = anndata.read_h5ad(fn)
    conddf = cell_meta[cell_meta['MajorType']==ct].loc[adata.obs_names]

    for obsm in [
#                 'X_pca_harmony', 
                 'X_pca_scanorama'
                ]:
        coorddf = pd.DataFrame(adata.obsm[obsm], index=adata.obs_names)
#         coorddf = coorddf.loc[conddf.index]
        n_pseudos = len(adata) if n_pseudos=='all' else n_pseudos

        sccee = SCCEE(conddf, coorddf, 0)
        rlt = sccee.evaluate_conditional_effect('Region', n_pseudos=n_pseudos, 
                                                n_nbrs=n_nbrs, use_cached_bg=True)
        
        tmp =rlt['pseudo_portion_matrix'].loc[adata.obs_names]
        radata = anndata.AnnData(tmp.values, obs=conddf, var=tmp.T[[]], )
        radata.obsm['X_pca'] = rlt['cond_embedding'].loc[adata.obs_names]    
        
        radata.write_h5ad(f'./regional-axes-data/{ct}.{supregs}.{obsm}.npseudos_{n_pseudos}.nnbrs_{n_nbrs}.h5ad')
    print('done')


L56_NP CX


/data/wtian/local/miniconda3/envs/sapiens/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -17.052792604175508 and the largest is 606.5037421189384.
  warn(
/gale/netapp/home/wtian/sapiens/systmp/ipykernel_85568/2159504037.py:33: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  radata = anndata.AnnData(tmp.values, obs=conddf, var=tmp.T[[]], )


done
L6b CX


/data/wtian/local/miniconda3/envs/sapiens/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -38.159440087664166 and the largest is 1974.8539145280718.
  warn(
/gale/netapp/home/wtian/sapiens/systmp/ipykernel_85568/2159504037.py:33: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  radata = anndata.AnnData(tmp.values, obs=conddf, var=tmp.T[[]], )


done
L6_CT CX


/data/wtian/local/miniconda3/envs/sapiens/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -49.23413341967836 and the largest is 2387.9312303524343.
  warn(


done


/gale/netapp/home/wtian/sapiens/systmp/ipykernel_85568/2159504037.py:33: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  radata = anndata.AnnData(tmp.values, obs=conddf, var=tmp.T[[]], )
